In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
from keras.datasets import mnist
import tensorflow as tf
import numpy as np
from random import randint
import keras
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers.merge import Concatenate
from keras.layers import Input, Reshape, Dense, Conv2D, Dropout, \
    MaxPooling2D, Flatten, UpSampling2D, Multiply, Activation, AveragePooling2D, \
    Add, Subtract, Lambda
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, TensorBoard
from scipy.ndimage import gaussian_filter as blur
from keras.optimizers import Adam
from keras.utils import multi_gpu_model as mgpu
from keras.utils import np_utils
import os, sys

Using TensorFlow backend.


In [2]:
imsize = 256
encspec = [
    [128],
    [256, 256, 256, 256], # 128
    [256, 256, 256, 256], # 64
    [256, 256, 256, 256], # 64
#     [256, 256, 256], # 32
    [512, 512, 512, 512],# 16
    [512, 512, 512, 512],# 16
#     [512, 512, 512], # 8
] # 4
poolspec = [ True, True, True, True, True, True ]

print('Conv Spec:')
dynsize = imsize
for lii,layerspec in enumerate(encspec):
    tosize = dynsize//2 if poolspec[lii] else dynsize
    if tosize % 2 == 1: tosize += 1
    print('%d => %d: %s' % (dynsize, tosize, layerspec))
    dynsize = tosize


Conv Spec:
256 => 128: [128]
128 => 64: [256, 256, 256, 256]
64 => 32: [256, 256, 256, 256]
32 => 16: [256, 256, 256, 256]
16 => 8: [512, 512, 512, 512]
8 => 4: [512, 512, 512, 512]


In [3]:
model = Sequential()
# NOTE: Initial layers needs an input_shape...
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', 
                 input_shape=(imsize, imsize, 1)))
for lii, layerspec in enumerate(encspec):
    for nFilters in layerspec:
        model.add(Conv2D(nFilters, (3, 3), padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
    if poolspec[lii]:
        model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='same')) # = 14
# always do additional conv after final maxpool
# for ii in range(4):
#     model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    
model.add(Reshape((dynsize * dynsize * 512,)))
model.add(Dense(2048))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(2048))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))


In [4]:
print(model.count_params())
model = mgpu(model)
opt = Adam(0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()


print('LR', K.eval(model.optimizer.lr))

45601034
LR 0.001


In [5]:
from scipy.ndimage import gaussian_filter as blur

nb_classes=10
(X_train, y_train), (X_test, y_test) = mnist.load_data()
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

def datagen(mode='train', bsize=64, zoom=2):
    XX, YY = X_train, Y_train
    if mode == 'test': XX, YY = X_test, Y_test
    while True:
        for bii in range(0, len(XX), bsize):
            imgs = XX[bii:bii+bsize]
            lbls = YY[bii:bii+bsize]
            
            heatmap = np.zeros((bsize, imsize, imsize, 1))
            inputs = np.zeros((bsize, imsize, imsize, 1))
            dsize = 28 * zoom
            for ii, im in enumerate(imgs):
                rx, ry = randint(0, imsize-dsize-1), randint(0, imsize-dsize-1)
                inputs[ii, ry:ry+dsize, rx:rx+dsize, 0] = \
                    cv2.resize(im.reshape((28, 28)), (0,0), fx=zoom, fy=zoom)
                heatmap[ii, ry:ry+dsize, rx:rx+dsize, 0] = \
                    cv2.resize(im.reshape((28, 28)), (0,0), fx=zoom, fy=zoom)
                heatmap[ii] = blur(heatmap[ii], 1)
                heatmap[ii, heatmap[ii] > 0] = 1
                heatmap[ii] = blur(heatmap[ii], 3)
            yield inputs, lbls
#             return inputs, lbls, heatmap

print(type(datagen()))
inputs, lbls = next(datagen())
print(inputs[0].shape, lbls[0].shape)

<class 'generator'>
(256, 256, 1) (10,)


In [6]:
# from keras.backend import categorical_crossentropy as crossent

# def mse(y_true, y_pred):
#     return K.mean(K.square(y_pred - y_true), axis=-1)
# # def logits(y_true, y_pred):
# #     return crossent(y_true, y_pred, from_logits=True)

# encoder = Model(images, [enc, y_preds])
# # encoder = Model(images, enc)
# # encoder.summary()
# encoder = mgpu(encoder)
# # encoder.compile('adam', mse)
# encoder.compile('adam', {
#     'y_preds': 'categorical_crossentropy',
#     'enc_preds': mse,
# })

# # encoder = Model(images, y_preds)
# # encoder = mgpu(encoder)
# # encoder.compile('adam', 'categorical_crossentropy')


In [7]:
bsize=32

LOGFOLDER = 'logs'
TAG = 'mnist'
try: shutil.rmtree('./%s/%s' % (LOGFOLDER, TAG)) 
except: pass
board_cb = TensorBoard('%s/%s' % (LOGFOLDER, TAG))

dset = datagen(bsize=bsize)
model.fit_generator(
    dset,
    epochs=4,
    verbose=1,
    steps_per_epoch=len(X_train)//bsize-1
)


Epoch 1/4
  33/1874 [..............................] - ETA: 25:37 - loss: 3.0249 - acc: 0.1875

KeyboardInterrupt: 